In [1]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

simpletransformers==0.63.4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS 487/Final project/ChineseHumorSentiment-master/data'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

['humor_category_train.txt', 'task2_development.csv', 'task1_train.csv', 'task1_development.csv', 'task1_test.csv', 'yinyu_type_train.txt', 'yinyu_senti_train.txt', 'humor_degree_train.txt', 'task2_test.csv', 'task2_train.csv']


In [4]:
import pandas as pd

In [5]:
data1 = pd.read_csv("drive/My Drive/EECS 487/Final project/ChineseHumorSentiment-master/data/task1_train.csv",encoding='utf8')
data2 = pd.read_csv("drive/My Drive/EECS 487/Final project/ChineseHumorSentiment-master/data/task2_train.csv",encoding='utf8')
data1 = data1.drop(columns = ['id'])
data2 = data2.drop(columns = ['id'])
data2['label'] = data2['label'].replace({1: 0, 3: 1, 5: 2})

In [6]:
data1.label.value_counts()

1    11536
0     4884
Name: label, dtype: int64

In [7]:
data2.label.value_counts()

1    8624
0    4408
2    3638
Name: label, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
train_df, test_df = train_test_split(data1)

In [11]:
print(train_df.shape)
print(test_df.shape)

(12315, 2)
(4105, 2)


In [12]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [13]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 2,
             "overwrite_output_dir": True
            }

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-chinese",
    num_labels=2,
    args=train_args
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/12315 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1540 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1540 [00:00<?, ?it/s]

(3080, 0.3878443897665042)

In [15]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4105 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/514 [00:00<?, ?it/s]

In [16]:
result

{'mcc': 0.5451056167792797,
 'tp': 2361,
 'tn': 916,
 'fp': 304,
 'fn': 524,
 'auroc': 0.886736085461829,
 'auprc': 0.9561963423051971,
 'f1': 0.7982947624847747,
 'acc': 0.7982947624847747,
 'eval_loss': 0.3852406832286368}

In [28]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args2 ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 2,
             "overwrite_output_dir": True
            }

# Create a ClassificationModel
model2 = ClassificationModel(
    "bert", "bert-base-chinese",
    num_labels=3,
    args=train_args2
)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
train_df, test_df = train_test_split(data2)

In [30]:
model2.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/12502 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1563 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/1563 [00:00<?, ?it/s]

(3126, 0.8958339655077083)

In [31]:
test_df.label.value_counts()

1    2150
0    1118
2     900
Name: label, dtype: int64

In [32]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model2.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4168 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/521 [00:00<?, ?it/s]

In [33]:
result

{'mcc': 0.2110186605639932,
 'f1': 0.5623800383877159,
 'acc': 0.5623800383877159,
 'eval_loss': 0.8751653788071447}

In [23]:
# import os
# import tarfile

# def pack_model(model_path='',file_name=''):
#   files = [files for root, dirs, files in os.walk(model_path)][0]
#   with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
#     for file in files:
#       f.add(f'{model_path}/{file}')

# # run the function
# pack_model('drive/My Drive/EECS 487/Final project/ChineseHumorSentiment-master/data/mBERT','mBERT')